# Problem: Implement a Deep Neural Network

### Problem Statement
You are tasked with constructing a **Deep Neural Network (DNN)** model to solve a regression task using PyTorch. The objective is to predict target values from synthetic data exhibiting a non-linear relationship.

### Requirements
Implement the `DNNModel` class that satisfies the following criteria:

1. **Model Definition**:
   - The model should have:
     - An **input layer** connected to a **hidden layer**.
     - A **ReLU activation function** for non-linearity.
     - An **output layer** with a single unit for regression.

<details> <summary>💡 Hint</summary> - Use `nn.Sequential` to simplify the implementation of the `DNNModel`. - Experiment with different numbers of layers and hidden units to optimize performance. - Ensure the final layer has a single output unit (since it's a regression task). </details> <details> <summary>💡 Bonus: Try Custom Loss Functions</summary> Experiment with custom loss functions (e.g., Huber Loss) and compare their performance with MSE. </details>

In [8]:
import torch
import torch.nn as nn
import torch.optim as optim

In [9]:
# Generate synthetic data
torch.manual_seed(42)
X = torch.rand(100, 2) * 10  # 100 data points with 2 features
y = (X[:, 0] + X[:, 1] * 2).unsqueeze(1) + torch.randn(100, 1)  # Non-linear relationship with noise

In [22]:
# Define the Deep Neural Network Model
class DNNModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(2,10),
            nn.ReLU(),
            nn.Linear(10, 1)
        )

    def forward(self, x):
        return self.layers(x)


class HuberLoss(nn.Module):
    def __init__(self, delta = 5.0):
        super().__init__()
        self.delta = delta
    def forward(self, y_preds, y_true):
        diff = abs(y_preds - y_true)
        loss = 0.5 * diff ** 2
        indices = [x for x in range(len(y_preds)) if abs(y_preds[x] - y_true[x]) > self.delta]
        loss[indices] = diff[indices] * self.delta + 0.5 * self.delta ** 2
        return loss.mean()

In [23]:
# Initialize the model, loss function, and optimizer
model = DNNModel()
criterion = HuberLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

# Training loop
epochs = 1000
for epoch in range(epochs):
    # Forward pass
    predictions = model(X)
    loss = criterion(predictions, y)

    # Backward pass and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Log progress every 100 epochs
    if (epoch + 1) % 100 == 0:
        print(f"Epoch [{epoch + 1}/{epochs}], Loss: {loss.item():.4f}")

# Testing on new data
X_test = torch.tensor([[4.0, 3.0], [7.0, 8.0]])
with torch.no_grad():
    predictions = model(X_test)
    print(f"Predictions for {X_test.tolist()}: {predictions.tolist()}")

Epoch [100/1000], Loss: 0.6625
Epoch [200/1000], Loss: 0.4735
Epoch [300/1000], Loss: 0.3991
Epoch [400/1000], Loss: 0.3690
Epoch [500/1000], Loss: 0.3576
Epoch [600/1000], Loss: 0.3538
Epoch [700/1000], Loss: 0.3520
Epoch [800/1000], Loss: 0.3505
Epoch [900/1000], Loss: 0.3494
Epoch [1000/1000], Loss: 0.3481
Predictions for [[4.0, 3.0], [7.0, 8.0]]: [[9.931657791137695], [23.093406677246094]]
